# 🚀 Quick Start Guide

**Before running:**
1. ✅ Upload `data/splits/*.jsonl` files to your Google Drive
2. ✅ Set `BASE_PATH` in cell 2 to your Google Drive folder
3. ✅ Set `YOUR_HF_USERNAME` in cell 8 to your Hugging Face username
4. ✅ Make sure you have a GPU runtime: Runtime > Change runtime type > T4 or A100

**Then:** Run all cells (Runtime > Run all)

**This notebook trains TWO models:**
- Model A: Context-Only approach
- Model B: Explain-and-Answer approach

**Expected time:** 2-6 hours total (depends on GPU)

# Task 2: Finetuning with Google Colab

**Goal:** Reproduce the training from `train.sh` using modern transformers and PEFT libraries.

**Two Experiments:**
- **Experiment A (Context-Only):** Fine-tune flan-t5-base on `train_context_only.jsonl` / `dev_context_only.jsonl`
- **Experiment B (Explain-and-Answer):** Fine-tune flan-t5-base on `train_exp_ans.jsonl` / `dev_exp_ans.jsonl`

**Note:** This notebook replaces the outdated `autotrain-advanced` approach with direct use of `transformers` and `peft` libraries, which work reliably in Google Colab.

---

## Time Tracking (for Reproducibility Log)
**Remember to log:**
- Start/End time for each experiment
- GPU type (Runtime > Change runtime type)
- GPU hours used
- Any errors or issues encountered

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set base path - ADJUST THIS to your Google Drive folder
BASE_PATH = "/content/drive/MyDrive/reproducing_project"

print(f"✓ Drive mounted. Base path: {BASE_PATH}")

Mounted at /content/drive


In [ ]:
# Install required libraries
!pip install -q transformers datasets peft accelerate bitsandbytes huggingface_hub

print("✓ All dependencies installed")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 139.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.3/341.3 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.9/269.9 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.5/225.5 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 12.4 MB/s eta 0

In [ ]:
# Prepare data directories for both experiments
import os

# Create directories
data_context_dir = f"{BASE_PATH}/data_context_only"
data_exp_ans_dir = f"{BASE_PATH}/data_exp_ans"

os.makedirs(data_context_dir, exist_ok=True)
os.makedirs(data_exp_ans_dir, exist_ok=True)

print("--- Preparing Experiment A (Context-Only) ---")
!cp "{BASE_PATH}/data/splits/train_context_only.jsonl" "{data_context_dir}/train.jsonl"
!cp "{BASE_PATH}/data/splits/dev_context_only.jsonl" "{data_context_dir}/valid.jsonl"
!ls -lh "{data_context_dir}/"

print("\n--- Preparing Experiment B (Explain-and-Answer) ---")
!cp "{BASE_PATH}/data/splits/train_exp_ans.jsonl" "{data_exp_ans_dir}/train.jsonl"
!cp "{BASE_PATH}/data/splits/dev_exp_ans.jsonl" "{data_exp_ans_dir}/valid.jsonl"
!ls -lh "{data_exp_ans_dir}/"

print("\n✓ Data prepared for both experiments")

--- Preparing data for Experiment A (Context-Only) ---
total 1.2M
-rw------- 1 root root 592K Nov  8 01:10 train.jsonl
-rw------- 1 root root 625K Nov  8 01:10 valid.jsonl

--- Preparing data for Experiment B (Explain-and-Answer) ---
total 1.4M
-rw------- 1 root root 643K Nov  8 01:10 train.jsonl
-rw------- 1 root root 700K Nov  8 01:10 valid.jsonl


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# Training function that matches train.sh configuration
import os
import time
from datetime import datetime
from datasets import load_dataset
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, TaskType

def train_model(experiment_name, data_path, model_name_suffix, your_hf_username):
    """
    Train a model matching the train.sh configuration.
    
    Args:
        experiment_name: "Context-Only" or "Explain-and-Answer"
        data_path: Path to training data directory
        model_name_suffix: Suffix for model name (e.g., "context-only")
        your_hf_username: Your Hugging Face username
    """
    print(f"\n{'='*60}")
    print(f"  EXPERIMENT: {experiment_name}")
    print(f"{'='*60}\n")
    
    start_time = time.time()
    start_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"⏱️  Start Time: {start_datetime}")
    
    # Configuration (matching train.sh)
    base_model_id = "google/flan-t5-base"
    new_model_repo = f"{your_hf_username}/flan-t5-{model_name_suffix}"
    
    # LoRA Config (from train.sh: use-peft with default LoRA settings)
    peft_config = LoraConfig(
        task_type=TaskType.SEQ_2_SEQ_LM,
        r=16,              # LoRA rank
        lora_alpha=32,     # LoRA alpha
        lora_dropout=0.05  # LoRA dropout
    )
    
    # Training Args (matching train.sh parameters)
    training_args = TrainingArguments(
        output_dir=f"{BASE_PATH}/models/{model_name_suffix}",
        learning_rate=2e-4,                    # from train.sh
        per_device_train_batch_size=4,         # from train.sh
        per_device_eval_batch_size=4,
        num_train_epochs=3,                    # from train.sh
        logging_steps=10,
        save_strategy="epoch",
        evaluation_strategy="epoch",
        load_best_model_at_end=True,
        push_to_hub=True,
        hub_model_id=new_model_repo,
        report_to="none",  # Disable wandb/tensorboard
        fp16=True,  # Enable mixed precision for faster training
    )
    
    print(f"📦 Loading dataset from: {data_path}")
    # Load dataset
    raw_datasets = load_dataset('json', data_files={
        'train': os.path.join(data_path, 'train.jsonl'),
        'validation': os.path.join(data_path, 'valid.jsonl')
    })
    
    print(f"   - Training examples: {len(raw_datasets['train'])}")
    print(f"   - Validation examples: {len(raw_datasets['validation'])}")
    
    print(f"\n🤖 Loading model and tokenizer: {base_model_id}")
    # Load model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(base_model_id)
    
    # Preprocessing function (matching train.sh: model_max_length=1024)
    def preprocess_function(examples):
        model_inputs = tokenizer(
            examples['input'], 
            max_length=1024,  # from train.sh
            truncation=True,
            padding=False  # Will pad dynamically
        )
        
        labels = tokenizer(
            examples['output'], 
            max_length=256,   # Reasonable output length
            truncation=True,
            padding=False
        )
        
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    
    print("🔄 Tokenizing dataset...")
    tokenized_datasets = raw_datasets.map(
        preprocess_function, 
        batched=True,
        remove_columns=raw_datasets["train"].column_names
    )
    
    print("🔧 Applying LoRA (PEFT)...")
    # Apply LoRA
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    
    # Data collator for dynamic padding
    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model,
        padding=True
    )
    
    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    
    print(f"\n🚀 Starting training for: {experiment_name}")
    print(f"   Model will be saved to: {new_model_repo}")
    
    # Train
    trainer.train()
    
    print(f"\n💾 Pushing model to Hugging Face Hub: {new_model_repo}")
    # Push to hub
    trainer.push_to_hub()
    
    # Calculate time
    end_time = time.time()
    end_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    duration_seconds = end_time - start_time
    duration_hours = duration_seconds / 3600
    
    print(f"\n{'='*60}")
    print(f"  ✅ EXPERIMENT COMPLETE: {experiment_name}")
    print(f"{'='*60}")
    print(f"⏱️  End Time: {end_datetime}")
    print(f"⏱️  Duration: {duration_hours:.2f} hours ({duration_seconds/60:.1f} minutes)")
    print(f"🔗 Model Hub: https://huggingface.co/{new_model_repo}")
    print(f"\n📝 LOG THIS IN REPRODUCIBILITY_LOG.md:")
    print(f"   - Experiment: {experiment_name}")
    print(f"   - Start: {start_datetime}")
    print(f"   - End: {end_datetime}")
    print(f"   - GPU Hours: {duration_hours:.2f}")
    print(f"   - Model: {new_model_repo}")
    print(f"{'='*60}\n")
    
    return trainer, duration_hours

print("✓ Training function defined")

---
## 🔬 Experiment A: Context-Only

Fine-tune on prompts that include only the conflicting contexts (no explanation requested).

In [ ]:
# ⚠️ IMPORTANT: Set your Hugging Face username here!
YOUR_HF_USERNAME = "YOUR_USERNAME_HERE"  # Replace with your actual username

# Run Experiment A
trainer_a, hours_a = train_model(
    experiment_name="Experiment A: Context-Only",
    data_path=f"{BASE_PATH}/data_context_only",
    model_name_suffix="context-only",
    your_hf_username=YOUR_HF_USERNAME
)

---
## 🔬 Experiment B: Explain-and-Answer

Fine-tune on prompts that ask the model to explain the conflict AND provide an answer.

In [ ]:
# Run Experiment B
trainer_b, hours_b = train_model(
    experiment_name="Experiment B: Explain-and-Answer",
    data_path=f"{BASE_PATH}/data_exp_ans",
    model_name_suffix="explain-and-answer",
    your_hf_username=YOUR_HF_USERNAME
)

---
## 📊 Summary

Total GPU hours and summary of both experiments.

In [ ]:
# Summary of both experiments
total_hours = hours_a + hours_b

print("="*70)
print("  📊 TRAINING SUMMARY")
print("="*70)
print(f"\nExperiment A (Context-Only):")
print(f"  - GPU Hours: {hours_a:.2f}")
print(f"  - Model: {YOUR_HF_USERNAME}/flan-t5-context-only")

print(f"\nExperiment B (Explain-and-Answer):")
print(f"  - GPU Hours: {hours_b:.2f}")
print(f"  - Model: {YOUR_HF_USERNAME}/flan-t5-explain-and-answer")

print(f"\n{'='*70}")
print(f"  TOTAL GPU HOURS: {total_hours:.2f}")
print(f"{'='*70}")

print("\n📝 Next steps:")
print("  1. Update REPRODUCIBILITY_LOG.md with these GPU hours")
print("  2. Check GPU type: Runtime > Change runtime type")
print("  3. Test the models on the test set")
print("  4. Document any issues encountered")